In [ ]:
import openpyxl
import pprint
import pandas as pd

In [22]:
import openpyxl
import pprint

#Excel読み込み
wb1 = openpyxl.load_workbook("sample.xlsx")
wb2 = openpyxl.load_workbook("seikyu.xlsx")
print(wb1.sheetnames)
print(wb2.sheetnames)

#シートネーム全部リスト化
wb1sheet = [sheetname for sheetname in wb1.sheetnames]
print(wb1sheet)
wb2sheet = [sheetname for sheetname in wb2.sheetnames]


sheet1 = wb1[wb1sheet[0]]
print(sheet1['A4'].value)
max1 = sheet1.max_row

col = [col for col in sheet1.rows() ]

def get_value_list(t_2d):
    return([[cell.value for cell in row] for row in t_2d])

l_2d = get_value_list(sheet1)

def get_list_2d(sheet, start_row, end_row, start_col, end_col):
    return get_value_list(sheet.iter_rows(min_row=start_row,
                                          max_row=end_row,
                                          min_col=start_col,
                                          max_col=end_col))

l_2d = get_list_2d(sheet1, 4, max1, 0, 8)

pprint.pprint(l_2d, width=40)





['請求一覧2110', 'Sheet3', '顧客管理テーブル']
['請求一覧2101', '請求一覧2102', '請求一覧2103', '請求一覧2105', '請求一覧2106', '請求一覧2107', '請求一覧2108', '請求一覧2109', '請求一覧2110', 'Sheet3', '顧客管理テーブル']
['請求一覧2110', 'Sheet3', '顧客管理テーブル']
210401
[[210401,
  None,
  3001,
  '納品管理システム',
  '変更作業',
  28,
  456000,
  None],
 [210601,
  None,
  4001,
  '次年度用職員募集',
  'ホームページ作成',
  10,
  350000,
  None],
 [210602,
  None,
  2001,
  'FK社名古屋工場向け',
  'ネットワーク設計',
  16,
  580000,
  None],
 [210701,
  None,
  1002,
  'SS 社向けプログラム開発',
  '(ss07534, ss07535)',
  20,
  648000,
  None],
 [210702,
  None,
  1001,
  'TN 社向け営業支援',
  None,
  20,
  133000,
  None],
 [210803,
  None,
  4001,
  '次年度用ホームページ',
  '仕様変更対応作業',
  22,
  55000,
  None],
 [210901,
  None,
  1001,
  'ＫＹ社向け営業支援',
  '継続活動',
  25,
  80000,
  '21年10月分'],
 [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None]]
